In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter

# Load corrected DEM
dem = np.load('../output/sfs/dem_shadow_corrected.npy')


In [ ]:
# Apply light smoothing
smoothed_dem = gaussian_filter(dem, sigma=2)

# Normalize to 0–1 for visualization
dem_min = np.nanmin(smoothed_dem)
dem_max = np.nanmax(smoothed_dem)
dem_norm = (smoothed_dem - dem_min) / (dem_max - dem_min)

plt.imshow(dem_norm, cmap='terrain')
plt.title("Smoothed & Normalized DEM")
plt.colorbar()
plt.show()

np.save('../output/final_dem/dem_normalized.npy', dem_norm)


In [ ]:
import rasterio
from rasterio.transform import from_origin
import cv2

# Save visual PNG
cv2.imwrite('../output/final_dem/dem_visual_final.png', (dem_norm * 255).astype('uint8'))

# Export to GeoTIFF (assuming pixel size = 1m, origin = (0,0))
transform = from_origin(0, 0, 1.0, 1.0)
with rasterio.open(
    '../output/final_dem/dem_normalized.tif', 'w',
    driver='GTiff',
    height=dem_norm.shape[0],
    width=dem_norm.shape[1],
    count=1,
    dtype='float32',
    crs='+proj=latlong',
    transform=transform
) as dst:
    dst.write(dem_norm.astype('float32'), 1)
